First off, this code is not great.

This code utilizes selenium, a webscraping tool, to pull archived
avalanche danger reports. An alternative tool, BeautifulSoup, is 
NOT used because it only scrapes HTML data and not the JavaScript
data that the Northwest Avalanche Center (NWAC) site uses for 
display. Selenium is a bit of a beast to work with unfortunately
and if another tool is available we should at least look into it.

This code opens and navigates to the NWAC archived avalanche reports
site and pulls the links to individual avalanche reports. Then these
pages are scraped to find danger ratings.

Initialize the selenium webdriver to scrape the page

In [1]:
import os
import time

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By

def init_driver():
    """Initialize the web driver (essentially 'open the webpage')"""
    # `snap install firefox`
    # --- 
    # Set up the webdriver. 
    # This code will likely be different depending on the user's computer. 
    install_dir = "/snap/firefox/current/usr/lib/firefox"
    driver_loc = os.path.join(install_dir, "geckodriver")
    binary_loc = os.path.join(install_dir, "firefox")

    service = Service(driver_loc)
    opts = Options()
    opts.add_argument('-headless') # comment out for testing
    opts.binary_location = binary_loc
    # ---
    driver = webdriver.Firefox(service=service, options=opts)
    return driver


Write some code to actually scrape the page. See associated video which verifies that the data is correct

In [2]:
def print_danger_ratings(url):
    # Make a new driver because reuse sometimes causes problems
    driver = init_driver()
    driver.get(url)
    time.sleep(2)
    # Find all elements with the specified class name
    res = driver.find_elements(By.CLASS_NAME, 'nac-dangerLabel')
    print('actual:')
    for danger in res:
        if danger.text:
            print('\t'+danger.text)

def test_danger_ratings(driver, url, expected):
    print('expected:')
    print(expected)
    print_danger_ratings(url)



def run_some_tests():
    test_danger_ratings(driver, 'https://nwac.us/avalanche-forecast/#/forecast/6/124162', \
                        '\tmoderate\n\tmoderate\n\tlow')
    print('---')
    test_danger_ratings(driver, 'https://nwac.us/avalanche-forecast/#/forecast/4/124286',\
                        '\tconsiderable\n\tconsiderable\n\tmoderate')
    print('---')
    test_danger_ratings(driver, 'https://nwac.us/avalanche-forecast/#/forecast/4/123859',\
                        '\thigh\n\thigh\n\tconsiderable')
    print('---')
    test_danger_ratings(driver, 'https://nwac.us/avalanche-forecast/#/forecast/2/123737',\
                        '\tmoderate\n\tmoderate\n\tmoderate')
    print('---')

#driver = init_driver()
#run_some_tests()



Get a bunch of data

In [3]:

# Get the link to the report page
# This is where selenium gets weird about navigating things...
# See documentation for more info, but it's finding it my XML path
"""
page = driver.find_elements(By.XPATH, "//a[contains(@href, '#/forecast/')]")
for p in page[15:20]:
    print('link:', p.get_attribute('href'))
    print_danger_ratings(p.get_attribute('href'))
"""

# todo: multiple pages

def get_danger_ratings(url):
    # Make a new driver because reuse sometimes causes problems
    driver = init_driver()
    driver.get(url)
    time.sleep(2)
    # Find all elements with the specified class name
    res = driver.find_elements(By.CLASS_NAME, 'nac-dangerLabel')
    valid = [d for d in res if d.text]
    dangers = [d.text.split(' ')[-1] for d in valid]
    driver.close()
    return dangers

def get_ratings(driver):

    danger_ratings = []

    def add_to_report(driver, ratings, first=False, start=0):
        # find the element that contains the archived reports
        report_div = driver.find_element(By.CLASS_NAME, 'nac-forecast-archive')
        table = report_div.find_element(By.CLASS_NAME, 'nac-html-table')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        trs = tbody.find_elements(By.TAG_NAME, 'tr')

        for tr in trs[(start if first else 0):]:
            td = tr.find_elements(By.TAG_NAME, 'td')
            # "parse" table row data
            date = td[0].find_element(By.TAG_NAME, 'a').text
            url = td[0].find_element(By.TAG_NAME, 'a').get_attribute('href')         
            location = td[3].text
            # print(date, location, url)
            try:
                danger_above, danger_at, danger_below = get_danger_ratings(url)
            except Exception as err:
                print(url, err)
                anger_above, danger_at, danger_below = 0, 0, 0
            # danger = td[5].find_element(By.TAG_NAME, 'span').text
            ratings.append((date, location, danger_above, danger_at, danger_below))

    page_count = 15

    # NWAC Archived Avalanche Reports from the 22-23 season
    # Provides daily danger predictions by area
    archive = 'https://nwac.us/avalanche-forecast/#/archive/'
    # "Get" the page
    driver.get(archive)
    # Wait for the page to load. There's probably a much better way to do this
    time.sleep(2)

    for i in range(page_count):
        add_to_report(driver, danger_ratings, first=(i == 0), start=50)
        next_page_button = driver.find_elements(By.XPATH, "//a[contains(text(), '>')]")[0]
        next_page_button.click()

    return danger_ratings


driver = init_driver()

danger_ratings = get_ratings(driver)

print(len(danger_ratings))
print(danger_ratings[100:105])

driver.close()

https://nwac.us/avalanche-forecast/#/forecast/6/123980 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/5/123796 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/9/123657 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/1/122905 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/4/122110 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/5/122028 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/5/121721 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/7/120339 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/4/119454 not enough values to unpack (expected 3, got 0)
https://nwac.us/avalanche-forecast/#/forecast/7/119231 not enough values 

What data do we actually need?
- Date
- Location
- Danger Prediction
    - Above, Near, Below Treeline - take max for now


In [4]:
import csv

file = open('danger_ratings.csv', 'w')
writer = csv.writer(file)
# BAD
# writer.writerow('Date,Area,Above Treeline,At Treeline,Below Treeline')
for rating in danger_ratings:
    writer.writerow(rating)
file.close()